# 论文引用网络节点分类比赛 Baseline

## 赛题介绍


图神经网络（Graph Neural Network）是一种专门处理图结构数据的神经网络，目前被广泛应用于推荐系统、金融风控、生物计算中。图神经网络的经典问题主要有三种，包括节点分类、连接预测和图分类三种。本次比赛是图神经网络7日打卡课程的大作业，主要让同学们熟悉如何图神经网络处理节点分类问题。

数据集为论文引用网络，图由大量的学术论文组成，节点之间的边是论文的引用关系，每一个节点提供简单的词向量组合的节点特征。我们的目的是给每篇论文推断出它的论文类别。





## 运行方式
本次基线基于飞桨PaddlePaddle 1.8.4版本，若本地运行则可能需要额外安装pgl、easydict、pandas等模块。

## 本地运行
下载左侧文件夹中的所有py文件（包括build_model.py, model.py）,以及work目录，然后在右上角“文件”->“导出Notebook到py”，这样可以保证代码是最新版本），执行导出的py文件即可。完成后下载submission.csv提交结果即可。

## AI Studio (Notebook)运行
依次运行下方的cell，完成后下载submission.csv提交结果即可。若运行时修改了cell，推荐在右上角重启执行器后再以此运行，避免因内存未清空而产生报错。 Tips：若修改了左侧文件夹中数据，也需要重启执行器后才会加载新文件。

## 代码整体逻辑

1. 读取提供的数据集，包含构图以及读取节点特征（用户可自己改动边的构造方式）

2. 配置化生成模型，用户也可以根据教程进行图神经网络的实现。

3. 开始训练

4. 执行预测并产生结果文件


## 环境配置
根据你进入的环境选择GPU版本的paddle或者是CPU版本的paddle，后面写运行脚本需要用到。<br/>
CPU的话下载飞桨paddlepaddle==1.8.5, 以及pgl==1.2.1。

In [1]:
# CPU
# !pip install paddlepaddle==1.8.5
# GPU
!pip install paddlepaddle-gpu
!pip install pgl==1.2.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 7.9MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 7.8MB/s eta 0:00:011
     |████████████████████████████████| 81kB 2.8MB/s eta 0:00:011


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [2]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd

## 图网络配置
使用最基础的版本

In [3]:
from easydict import EasyDict as edict
#easydict这个模块下的EasyDict，可以使得创建的字典像访问属性一样
config = {
    "model_name": "GCN",
    "num_layers": 1,    # 网络层数--这个实现在模型类的forward里边，通过循环实现
    "dropout": 0.5,     # 训练时，参数drop概率
    "learning_rate": 0.01,    # 训练优化的学习率
    "weight_decay": 0.0005,   # 权重正则化率
    "edge_dropout": 0.00,     # 边drop概率
}

config = edict(config)  # 利用EasyDict便利字典的读取

## 数据加载模块

这里主要是用于读取数据集，包括读取图数据构图，以及训练集的划分。

### 1.边数据的加载与处理

In [4]:
from collections import namedtuple
#定义一个元组类型的Dataset,其中有[]中这些属性。
Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])
#加载数据
def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    '''
        input:
            num_nodes: 节点数
            self_loop: 是否加载自环边
            add_inverse_edge: 是否添加反转的边--即对应无向图的情况
    '''
    # 从数据中读取边
    edges = pd.read_csv("work/edges.csv", header=None, names=["src", "dst"]).values
    #如果需要添加反向边的话就在其中添加反向边
    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])
        # vstack沿竖直方向拼接--如：A =[1, 2] , B = [2, 3]; vstack([A, B]) => [[1, 2], [2, 3]]
        # eg: edges=[[1, 3], [2, 5], [6, 7]] => edges[:, ::-1]=[[3, 1], [5, 2], [7, 6]]
        # 再拼接就得到了正反边的一个集合了
    #添加自环边
    if self_loop:
        src = np.arange(0, num_nodes) # 定义n个节点作为起点，其中起点为0，终点为num_nodes
        dst = np.arange(0, num_nodes)# 定义n个节点作为终点--且与src一一对应
        self_loop = np.vstack([src, dst]).T# 再将两个行向量拼接(此时shape:[2, num_nodes]), 然后再转置T=>得到shape:[num_node, 2]这是的数据0->0， 1->1 ...就得到了自环边的数据
        edges = np.vstack([edges, self_loop])# 将自环边数据添加到本身的边数据中
    
    return edges

### 2.数据的完整加载与处理

In [5]:
def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("work/feat.npy") # 读取节点特征--每个节点100个特征
    #shape(node_feat)=130644, 100
    num_nodes = node_feat.shape[0]
    # print(node_feat.shape[0],node_feat.shape[1])
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)  # 根据实际传入的节点数，返回合理的边--这里包含自环边以及正向和反向的边
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat}) # 创建图：节点数、边数据、以及节点特征的字典
    
    # 计算当前图的所有节点的入度--返回一个list==>等价于graph.indegree(nodes=None)，nodes指定，返回指定的入度
    indegree = graph.indegree()
    # print(indegree)
    # 返回取最大入度中的一个,最低入度为1
    norm = np.maximum(indegree.astype("float32"), 1)
    # print(norm)
    # 利用这个最大入读计算一个归一化参数
    norm = np.power(norm, -0.5)
    # 将归一化参数添加到节点的norm特征中, shape[1], 只含有一个元素的序列，但不算标量：如，a 和 [a]
    graph.node_feat["norm"] = np.expand_dims(norm, -1)
    # 读取总的训练数据
    df = pd.read_csv("work/train.csv")
    # 读取总的节点的索引序列（集）
    node_index = df["nid"].values
    # 读取总的节点的label序列
    node_label = df["label"].values
    # print(len(node_index))
    # 70235
    # 划分训练数据集--80%--这里是计算一个训练集数目值
    train_part = int(len(node_index) * 0.8)
    # 利用训练集数目进行划分--0:train_part
    train_index = node_index[:train_part]
    # 训练label划分
    train_label = node_label[:train_part]
    # 验证数据valid_index划分
    valid_index = node_index[train_part:]
    # 验证valid_label划分
    valid_label = node_label[train_part:]
    # 读取测试集--也就是赛题提交数据--指定读取['nid']列数据
    test_index = pd.read_csv("work/test.csv")["nid"].values
    # 这是一个可以使用名字来访问元素内容的dataset,直接对应的属性载入。
    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    return dataset

### 数据的读取与分割

In [6]:
dataset = load()# 执行load函数获取完整的dataset（可命名索引的tuple）数据

# 从dataset中读取出相应数据
train_index = dataset.train_index    # 读取训练索引序列
train_index = np.expand_dims(train_index, -1)   # 在最后一位添加一个维度，保证数据向量化[[a]]
train_label = np.reshape(dataset.train_label, [-1 , 1])  # 读取训练label序列

# 对验证集进行操作
val_index = dataset.valid_index
val_index = np.expand_dims(val_index, -1)
val_label = np.reshape(dataset.valid_label, [-1, 1])

#对测试集进行操作
test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
#初始化一个数组，用于保存最终结果--提前用zeros创建一个空白矩阵，并指明数据类型
test_label = np.zeros((len(test_index), 1), dtype="int64")

## 组网模块

这里是组网模块，目前已经提供了一些预定义的模型，包括**GCN**, **GAT**, **APPNP**等。可以通过简单的配置，设定模型的层数，hidden_size等。你也可以深入到model.py里面，去奇思妙想，写自己的图神经网络。

In [7]:
import pgl
import model
import paddle.fluid as fluid
import numpy as np
import time
from build_model import build_model


# fluid.CUDAPlace(0)
# fluid.CPUPlace()
use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()


# 创建主program  -- paddle静态图都是在相应的program中运行的 -- 通常为start
train_program = fluid.default_main_program()
# 创建start_program -- 是我们运行的开始
startup_program = fluid.default_startup_program()
#配置执行器执行空间(block)的操作部分
#program_guard接口配合使用python的 with 语句来将 with block 里的算子和变量添加进指定的全局主程序（main program）和启动程序（startup program）。
# 以下执行的算子等都会放入到train_program->startup_program的block中
with fluid.program_guard(train_program, startup_program):
    # 开启一个命名空间--常用program_guard一起使用
    with fluid.unique_name.guard():
        # 这里使用到build_model.py中的函数，执行模型和参数的配置，并返回相关的data变量
        # 这个过程的算子都会被记录到（train_program, startup_program）对应的工作空间中
        gw, loss, acc, pred = build_model(dataset,
                            config=config,
                            phase="train",
                            main_prog=train_program)
# 创建一个新的Program作为test_program
test_program = fluid.Program()
# 含义如上，这里是开启(test_program, startup_program)的工作空间，并记录相应的算子、变量
with fluid.program_guard(test_program, startup_program):
    # 开启一个命名空间
    with fluid.unique_name.guard():
        # 返回test的模型参数等 
        _gw, v_loss, v_acc, v_pred = build_model(dataset,
                            config=config,
                            phase="test",
                            main_prog=test_program)

# 总结——program_guard确定工作环境--unique_name开启一个相应的命名空间，相辅相成。
# 克隆test_program
test_program = test_program.clone(for_test=True)
# 创建一个解释器
exe = fluid.Executor(place)


## 开始训练过程

图神经网络采用FullBatch的训练方式，每一步训练就会把所有整张图训练样本全部训练一遍。<br/>

训练轮次选为2000轮

In [ ]:
# 训练轮次
epoch = 2000
# 执行器运行-->优先执行
exe.run(startup_program)

# 将图数据变成 feed_dict 用于传入Paddle Excecutor
# 图数据原型：graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})  
# 创建图：节点数、边数据、以及节点特征的字典
# 调用to_feed方法，将图数据转换为feed_dict，用于执行器的输入参数
feed_dict = gw.to_feed(dataset.graph)
# 训练开始


for epoch in range(epoch):
    # Full Batch 训练 ==  单batch_size训练--全数据一次投入
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    # 往feed_dict中添加键值对数据--每一个轮次数据都会重新赋值更新
    feed_dict["node_index"] = np.array(train_index, dtype="int64")
    feed_dict["node_label"] = np.array(train_label, dtype="int64")
    # 执行器执行--执行train_program这个program空间内的算子和参数
    # 传入的数据：graph..., node_index, node_label
    # 需要计算返回的数据
    # 返回numpy数据
    train_loss, train_acc = exe.run(train_program,
                                feed=feed_dict,
                                fetch_list=[loss, acc],
                                return_numpy=True)
    # Full Batch 验证
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    # 执行器执行--执行test_program这个program空间内的算子和参数
    # 传入的数据：graph..., node_index, node_label
    # 需要计算返回的数据
    # 返回numpy数据
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    # 打印训练数据
    print("Epoch", epoch, "Train Acc", train_acc[0], "Valid Acc", val_acc[0])

W1103 15:08:32.368480   129 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 10.1, Runtime API Version: 9.0
W1103 15:08:32.372081   129 device_context.cc:260] device: 0, cuDNN Version: 7.6.


Epoch 0 Train Acc 0.038976293 Valid Acc 0.14843027
Epoch 1 Train Acc 0.14949812 Valid Acc 0.2301559
Epoch 2 Train Acc 0.19037873 Valid Acc 0.282765
Epoch 3 Train Acc 0.2336086 Valid Acc 0.2534349
Epoch 4 Train Acc 0.24302343 Valid Acc 0.25122803
Epoch 5 Train Acc 0.24361074 Valid Acc 0.28013098
Epoch 6 Train Acc 0.2664626 Valid Acc 0.32868227
Epoch 7 Train Acc 0.28918988 Valid Acc 0.40784508
Epoch 8 Train Acc 0.29470706 Valid Acc 0.41510642
Epoch 9 Train Acc 0.29185948 Valid Acc 0.41581833
Epoch 10 Train Acc 0.3258703 Valid Acc 0.3872713
Epoch 11 Train Acc 0.34788567 Valid Acc 0.35495123
Epoch 12 Train Acc 0.3367979 Valid Acc 0.33914715
Epoch 13 Train Acc 0.3326155 Valid Acc 0.33039084
Epoch 14 Train Acc 0.33104932 Valid Acc 0.3251228
Epoch 15 Train Acc 0.32631522 Valid Acc 0.32348543
Epoch 16 Train Acc 0.32619065 Valid Acc 0.3264042
Epoch 17 Train Acc 0.32821956 Valid Acc 0.33352318
Epoch 18 Train Acc 0.33788353 Valid Acc 0.3442728
Epoch 19 Train Acc 0.34450418 Valid Acc 0.36021927
Ep

## 对测试集进行预测

训练完成后，我们对测试集进行预测。预测的时候，由于不知道测试集合的标签，我们随意给一些测试label。最终我们获得测试数据的预测结果。


In [8]:
feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #存储的假标签，一开始全是0
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]

NameError: name 'feed_dict' is not defined

## 生成提交文件

最后一步，我们可以使用pandas轻松生成提交文件，最后下载 submission.csv 提交就好了。

In [ ]:
submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission.csv", index=False)

##  总结
1.执行器和解释器不能开同样的两回，不然会报错。<br/>
2.GPU的解释器比CPU的解释器快多了。有GPU训练太幸福了。<br/>
3.这次版本就是调了训练次数，等熟悉多了再对GCN模型进行改进。原本想尝试十折交叉验算来提高准确度的，但是最后没调通。<br/>